**REVISIÓN DE CONDICIONES PARA PARTICIPAR EN INSTAGRAM SOY UN POLLITO**

**Instrucciones**
- Seguirme en Instagram
- Escribir en comentario: Soy un pollito 🐥
- Si etiquetas hasta 5 personas tienes más posibilidades de ganar
- Si las personas etiquetadas me siguen tienes aún más posibilidades de ganar

**Librerias**

In [1]:
import numpy as np
import pandas as pd
import gspread as gs
import gspread_dataframe as gd

import google.colab.auth as auth
from google.colab import auth

In [2]:
from google.colab import auth
auth.authenticate_user()

from google.auth import default
creds, _ = default()

In [3]:
# Cliente
client = gs.authorize(creds)

In [4]:
# Leer mi spreadsheet
spreadsheet = client.open_by_key('1LIoRzpLFIcSvZPnyy7qSZDYhVOrIzhqGndSD81u9PJE')

In [5]:
# Leer sheets (hojas)
sheet1 = spreadsheet.worksheet('followers')
sheet2 = spreadsheet.worksheet('comments')

In [6]:
# Convertir en dataframes
df_sheet1 = gd.get_as_dataframe(sheet1)
df_sheet2 = gd.get_as_dataframe(sheet2)

**Analizar datos**

**1. Seguidores**

In [13]:
followers = (
    df_sheet1
    .loc[:, ['followers']]
    .query('followers.notnull()')
    .query('~followers.str.contains(r"AM$|PM$")')
    .rename(columns={'followers': 'usuario'})
)

In [14]:
followers.head()

,usuario
0,mich.garcia09
2,amnauj_408
4,ivandc755
6,alfredo.flores.2
8,dahiwilliams


**2. Comentarios**

In [23]:
comments = (
    df_sheet2
    .loc[:, ['comentario']]
    .query('comentario.notnull()')
    .assign(comentario = lambda x: x['comentario'].str.lower())
    .query('~comentario.str.contains("translation|replies|reply")')
    .assign(
        lead = lambda x: x['comentario'].shift(-1)
    )
    .assign(lead = lambda x: x['lead'].astype(str))
    .groupby(['comentario'], dropna=False)
    .agg(lead=('lead', ' '.join))
    .reset_index()
    .iloc[::2]
    .loc[:, ['comentario', 'lead']]
    .rename(columns={
        'comentario': 'usuario', 'lead': 'comentario'
    })
    .query('comentario.str.contains("🐤|🐣|🐥")')
    .sort_values(['usuario'])
)

In [26]:
comments.head()

,usuario,comentario
0,*soy un pollazo* 🐥,*soy un pollito* 🐥
2,0502chriss,soy un pollito 🐥 @quintanarojasbrenda @brandon...
120,_alfajorcito,soy un pollito 🐥
124,_gianpier5,soy un pollito 🐥@montezayda @ioove4 @rociomont...
126,_rockemma,soy un pollito 🐥


In [29]:
comments.query("usuario=='chelinchelo_'")

,usuario,comentario
202,chelinchelo_,soy un pollito🐥 @axndrezin @lukkdarkk @luiggid...


**Verificar que personas que dejaron comentario me siguen en Instagram**

In [ ]:
validacion = (
    comments
    .merge(followers.assign(seguidor='Si'), how='left', on=['usuario'])
    .query('seguidor.notnull()')
    .drop(['seguidor'], axis=1)
    .assign(comentario = lambda x: x['comentario'].str.strip())
    .assign(comentario = lambda x: x['comentario'].str.replace(r'\s+', ' ', regex=True))
    .query('comentario.str.contains("soy un pollito")')
    .assign(arrobas = lambda x: x['comentario'].str.count('@')+1)
)

In [ ]:
validacion

,usuario,comentario,arrobas
0,aleeejzzk,soy un pollito 🐥,1
1,alejandroml.uwu,soy un pollito 🐥 @messhee,2
2,alejandropaglez,soy un pollito🐥,1
3,alessoloza,soy un pollito 🐥,1
4,angelramirezgomero,soy un pollito 🐥,1
5,brandongarro0,soy un pollito 🐣,1
6,cafe_dormido,soy un pollito 🐤🐣 @_eleinemh97 @yuli.huamani @...,6
7,carla.fig,soy un pollito 🐥,1
8,castillovicky72,soy un pollito 🐥🐥🐣 @gesica_rocibel @ana.agente...,6
9,chaska08,soy un pollito 🐥,1


In [ ]:
# prompt: from dataframe validacion i have to extract all the strings that with @ and store them in a column, these should be shown in rows with the usuario column

import pandas as pd
import re

def extract_mentions(df):
    """
    Extracts mentions (@username) from the 'comentario' column of a DataFrame.

    Args:
        df: pandas DataFrame with a 'comentario' column.

    Returns:
        pandas DataFrame with an added 'mentions' column containing a list of mentions.
    """

    def find_mentions(text):
        # Find all occurrences of @ followed by alphanumeric characters or underscores
        mentions = re.findall(r'@([a-zA-Z0-9_]+)', text)
        return mentions

    df['mentions'] = df['comentario'].apply(find_mentions)
    return df

# Assuming 'validacion' DataFrame is already created as in the provided code
validacion = extract_mentions(validacion)

# Explode the 'mentions' list to separate rows for each mention
validacion = validacion.explode('mentions')

In [ ]:
validacion_mentions_seguidores = (
    validacion
    .loc[:, ['usuario', 'mentions']]
    .drop_duplicates()
    .merge(followers.rename(columns={'usuario': 'seguidor'}), how='left', left_on='mentions', right_on='seguidor')
    .assign(arrobas_seguidores = lambda x: np.where(x['seguidor'].notnull(), 1, 0))
    .groupby(['usuario'], dropna=False)
    .agg(arrobas_seguidores = ('arrobas_seguidores', 'sum'))
    .reset_index()
)

In [ ]:
validacion_final = (
    validacion
    .merge(validacion_mentions_seguidores, how='left', on=['usuario'])
    .assign(total_arrobas = lambda x: x['arrobas'] + x['arrobas_seguidores'])
    .drop(['mentions','arrobas','arrobas_seguidores'], axis=1)
    .drop_duplicates()
)

In [ ]:
# Repeat 'usuario' based on 'arrobas'
df_repeated = []
for index, row in validacion_final.iterrows():
  for _ in range(int(row['total_arrobas'])):
    df_repeated.append(row)
df_repeated = pd.DataFrame(df_repeated)

**Input para el 23/11**

In [ ]:
df_repeated['usuario']

,usuario
0,aleeejzzk
1,alejandroml.uwu
1,alejandroml.uwu
1,alejandroml.uwu
2,alejandropaglez
...,...
57,vivianadelvallevilela
58,yohanazabalaga
59,ysa_montoya
59,ysa_montoya
